In [1]:
import cv2 as opencv
import numpy
import matplotlib.pyplot as plt
import tensorflow
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
plt.rcParams["figure.figsize"]=[7,7]

In [ ]:
def RescaleImage(frame, scale=0.5):
    width=int(frame.shape[1]*scale)
    height=int(frame.shape[0]*scale)
    dimension=(width, height)
    return opencv.resize(frame, dimension, interpolation=opencv.INTER_AREA)


In [ ]:
img=opencv.imread("./Datasets/Convolutional_Neural_Networks/dataset/single_prediction/cat_or_dog_2.jpg", opencv.IMREAD_GRAYSCALE)
img=opencv.resize(img, (80,80), interpolation=opencv.INTER_AREA)

In [ ]:
#img_f=opencv.medianBlur(img, 3)
#img_f=opencv.medianBlur(img_f, 3)

img_f=opencv.GaussianBlur(img, (5,5), opencv.BORDER_DEFAULT)

In [ ]:
canny=opencv.Canny(img_f, 25, 130)
img_s=canny

In [ ]:
img_pro=numpy.uint8(numpy.absolute(canny))

In [ ]:
#thresh=opencv.resize(img_f, (64,64), interpolation=opencv.INTER_AREA)
plt.imshow(img_s, cmap="gray")

In [ ]:
#lap=opencv.Laplacian(img, opencv.CV_64F)

sobelx=opencv.Sobel(img, opencv.CV_64F, 1, 0)
sobely=opencv.Sobel(img, opencv.CV_64F, 0, 1)
sobelxy=opencv.bitwise_or(sobelx, sobely)
img_pro=numpy.uint8(numpy.absolute(sobelxy))
threshold, thresh=opencv.threshold(img_pro, 10,120, opencv.ADAPTIVE_THRESH_MEAN_C)
sobelx=opencv.Sobel(thresh, opencv.CV_64F, 1, 0)
sobely=opencv.Sobel(thresh, opencv.CV_64F, 0, 1)
sobelxy=opencv.bitwise_or(sobelx, sobely)
#canny=opencv.Canny(img, 20, 100)
threshold, thresh=opencv.threshold(sobelxy, 0,255,opencv.THRESH_BINARY_INV)



In [ ]:
thresh=opencv.resize(sobelxy, (64,64), interpolation=opencv.INTER_AREA)
plt.imshow(thresh, cmap="gray")

In [ ]:
img=opencv.imread("./Datasets/Convolutional_Neural_Networks/dataset/single_prediction/cat_or_dog_2.jpg", opencv.IMREAD_GRAYSCALE)
img=opencv.resize(img, (80,80), interpolation=opencv.INTER_AREA)
img_f=opencv.GaussianBlur(img, (5,5), opencv.BORDER_DEFAULT)
canny=opencv.Canny(img_f, 25, 130)

In [ ]:
tag="cat"
images=4000
for i in range(images):
    path_read="./Datasets/Convolutional_Neural_Networks/dataset/training_set/"+tag+"s/"+tag+".{}.jpg".format(i+1)
    img=opencv.imread(path_read)
    if type(img)==None:
        print("End")
    else:
        img=opencv.resize(img, (32,32), interpolation=opencv.INTER_AREA)
        #img_r=opencv.GaussianBlur(img, (3,3), opencv.BORDER_DEFAULT)
        #canny=opencv.Canny(img_r, 25, 130)
        #img_pro=numpy.uint8(numpy.absolute(canny))
        path_write="./CNNtrain/"+tag+"s/"+tag+".{}.jpg".format(i+1)
        opencv.imwrite(path_write,img)
        if (i+1)%500==0:
            print("imagenes procesadas: {}".format(i+1))

In [12]:
X_train=numpy.zeros((8000,32,32,3), dtype=float)
y_train=numpy.zeros((8000,2), dtype=int)
images=4000
for i in range(images):
    path="./CNNtrain/cats/cat.{}.jpg".format(i+1)
    img=opencv.imread(path)
    #img_r=thresh.reshape(64*64)
    #img=numpy.uint8(numpy.absolute(img/255))
    #img=img.reshape(32,32,1)
    X_train[i]=img/255.
    y_train[i]=[0,1]
    if (i+1)%500==0:
        print("imagenes procesadas: {}".format(i+1))
for i in range(images):
    path="./CNNtrain/dogs/dog.{}.jpg".format(i+1)
    img=opencv.imread(path)
    #img_r=thresh.reshape(64*64)
    #img=numpy.uint8(numpy.absolute(img/255))
    #img=img.reshape(32,32,1)
    X_train[i+images]=img/255.
    y_train[i+images]=[1,0]
    if (i+1)%500==0:
        print("imagenes procesadas: {}".format(i+1+images))


imagenes procesadas: 500
imagenes procesadas: 1000
imagenes procesadas: 1500
imagenes procesadas: 2000
imagenes procesadas: 2500
imagenes procesadas: 3000
imagenes procesadas: 3500
imagenes procesadas: 4000
imagenes procesadas: 4500
imagenes procesadas: 5000
imagenes procesadas: 5500
imagenes procesadas: 6000
imagenes procesadas: 6500
imagenes procesadas: 7000
imagenes procesadas: 7500
imagenes procesadas: 8000


In [13]:
X_train.shape

(8000, 32, 32, 3)

In [14]:
input_shape=(32,32,3)

In [15]:
X_train = X_train.reshape(-1, 32, 32,3)

In [16]:
X_train.shape

(8000, 32, 32, 3)

In [27]:
droprate=0.1
Dropout=tensorflow.keras.layers.Dropout

cnn=tensorflow.keras.models.Sequential()
cnn.add(tensorflow.keras.layers.Convolution2D(filters=64, kernel_size=3, padding="same", activation="relu", input_shape=input_shape))
cnn.add(Dropout(droprate))

cnn.add(tensorflow.keras.layers.MaxPooling2D(pool_size=2, strides=2))
cnn.add(Dropout(droprate))

cnn.add(tensorflow.keras.layers.Convolution2D(filters=32, kernel_size=3, padding="same", activation="relu"))
cnn.add(Dropout(droprate))

cnn.add(tensorflow.keras.layers.MaxPooling2D(pool_size=2, strides=2))
cnn.add(Dropout(droprate))

cnn.add(tensorflow.keras.layers.Flatten())
cnn.add(Dropout(droprate))

cnn.add(tensorflow.keras.layers.Dense(units=128, activation="relu"))
cnn.add(Dropout(droprate))

cnn.add(tensorflow.keras.layers.Dense(units=64, activation="relu"))
cnn.add(Dropout(droprate))

#cnn.add(tensorflow.keras.layers.Dense(units=32, activation="sigmoid"))

cnn.add(tensorflow.keras.layers.Dense(units=2, activation="sigmoid"))
#cnn.add(Dropout(droprate))

In [28]:
opt = SGD(lr=0.0002, momentum=0.8)
cnn.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
cnn.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
dropout_12 (Dropout)         (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 16, 16, 32)        18464     
_________________________________________________________________
dropout_14 (Dropout)         (None, 16, 16, 32)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 8, 8, 32)         

In [29]:
cnn.fit(X_train, y_train,validation_data=(X_test, y_test), batch_size=16, epochs=20, verbose=1, shuffle=True)

Epoch 1/20
500/500 [==============================] - 28s 53ms/step - loss: 0.6941 - accuracy: 0.5160 - val_loss: 0.6553 - val_accuracy: 0.6345
Epoch 2/20
500/500 [==============================] - 26s 52ms/step - loss: 0.6451 - accuracy: 0.6370 - val_loss: 0.5975 - val_accuracy: 0.6880
Epoch 3/20
500/500 [==============================] - 26s 51ms/step - loss: 0.6039 - accuracy: 0.6724 - val_loss: 0.5711 - val_accuracy: 0.7205
Epoch 4/20
500/500 [==============================] - 26s 51ms/step - loss: 0.5703 - accuracy: 0.7057 - val_loss: 0.5546 - val_accuracy: 0.7230
Epoch 5/20
500/500 [==============================] - 26s 52ms/step - loss: 0.5368 - accuracy: 0.7288 - val_loss: 0.5262 - val_accuracy: 0.7350
Epoch 6/20
500/500 [==============================] - 26s 52ms/step - loss: 0.5172 - accuracy: 0.7415 - val_loss: 0.5303 - val_accuracy: 0.7370
Epoch 7/20
500/500 [==============================] - 27s 53ms/step - loss: 0.5064 - accuracy: 0.7478 - val_loss: 0.5269 - val_accuracy:

KeyboardInterrupt: 

In [ ]:
cnn.predict(X_train, verbose=1)

In [ ]:
vec=[]
for p in pred:
    vec.append(1*(pred[0][0]>.33))

In [ ]:
y_train

In [ ]:
i=1
img=opencv.imread("./Datasets/Convolutional_Neural_Networks/dataset/single_prediction/cat_or_dog_{}.jpg".format(i))
img=opencv.resize(img, (32,32), interpolation=opencv.INTER_AREA)
#img_r=opencv.GaussianBlur(img, (3,3), opencv.BORDER_DEFAULT)
#canny=opencv.Canny(img_r, 25, 130)
path="cat_or_dog.{}.jpg".format(i)
opencv.imwrite(path,img)

In [ ]:
X_s_p=numpy.zeros((2,32,32,1), dtype=int)
images=2
for i in range(images):
    path="cat_or_dog.{}.jpg".format(i+1)
    img=opencv.imread(path, opencv.IMREAD_GRAYSCALE)
    #img=numpy.uint8(numpy.absolute(img))
    #img_r=thresh.reshape(64*64)
    img=img.reshape(32,32,1)
    X_s_p[i]=img
    print("imagenes procesadas: {}".format(i+1))

In [ ]:
X_s_p.shape

In [ ]:
X_s_p = X_s_p.reshape(-1, 32, 32, 1)

In [ ]:
pred=cnn.predict(X_s_p/255, verbose=1)

In [ ]:
pred

In [ ]:
#test

In [7]:
tag="dog"
images=1000
for i in range(images):
    path_read="./Datasets/Convolutional_Neural_Networks/dataset/test_set/"+tag+"s/"+tag+".{}.jpg".format(i+1+4000)
    img=opencv.imread(path_read)
    if type(img)==None:
        print("End")
    else:
        img=opencv.resize(img, (32,32), interpolation=opencv.INTER_AREA)
        #img_r=opencv.GaussianBlur(img, (3,3), opencv.BORDER_DEFAULT)
        #canny=opencv.Canny(img_r, 25, 130)
        #img_pro=numpy.uint8(numpy.absolute(canny))
        path_write="./CNNtest/"+tag+"s/"+tag+".{}.jpg".format(i+1)
        opencv.imwrite(path_write,img)
        if (i+1)%500==0:
            print("imagenes procesadas: {}".format(i+1))

imagenes procesadas: 500
imagenes procesadas: 1000


In [11]:
X_test=numpy.zeros((2000,32,32,3), dtype=float)
y_test=numpy.zeros((2000,2), dtype=int)
images=1000
for i in range(images):
    path="./CNNtest/cats/cat.{}.jpg".format(i+1)
    img=opencv.imread(path)
    #img_r=thresh.reshape(64*64)
    #img=numpy.uint8(numpy.absolute(img/255))
    #img=img.reshape(32,32,1)
    X_test[i]=img/255.
    y_test[i]=[0,1]
    if (i+1)%500==0:
        print("imagenes procesadas: {}".format(i+1))
for i in range(images):
    path="./CNNtest/dogs/dog.{}.jpg".format(i+1)
    img=opencv.imread(path)
    #img_r=thresh.reshape(64*64)
    #img=numpy.uint8(numpy.absolute(img/255))
    #img=img.reshape(32,32,1)
    X_test[i+images]=img/255.
    y_test[i+images]=[1,0]
    if (i+1)%500==0:
        print("imagenes procesadas: {}".format(i+1+images))


imagenes procesadas: 500
imagenes procesadas: 1000
imagenes procesadas: 1500
imagenes procesadas: 2000


In [21]:
cnn.predict(X_test, verbose=1)

63/63 [==============================] - 1s 9ms/step


array([[0.56224024, 0.46205482],
       [0.52393687, 0.6892016 ],
       [0.2506603 , 0.83746433],
       ...,
       [0.34720844, 0.6263388 ],
       [0.6358118 , 0.20473915],
       [0.43773472, 0.37565863]], dtype=float32)